# Aggregate by Climate Zones

In [1]:
import numpy as np
import matplotlib as plt
import xarray as xr
import pandas as pd


def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    except AttributeError:
        return False
    return True



## File Control

######################################################
#
# File Control
#

working_directory  = "./"

thredds_root_dir   = "/projects/ECEP/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/"
thredds_root_dir   = "http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Original_CONUS/" # historical/LOCA2-CONUS___ACCESS-CM2.r1i1p1f1___historical.nc"


loca2_mod_member_file = "./LOCA2_Model_Member_Available_List.csv"

masking_file          = "./LOCA2_MASKS.nc"

masking_variable      = "LOCA2_CLIMDIV"


Rank      = 1

Scenarios = ["historical",
                 "ssp245",
                 "ssp370",
                 "ssp585"]

Vars = ["tasmax",
        "tasmin",
            "pr"]

VarI = ["X",
        "N",
        "P"]

#
######################################################

## LOCA2 Files

######################################################
#
# Pull LOCA2 List
#

avail_models_and_members_list = pd.read_csv(filepath_or_buffer = working_directory + loca2_mod_member_file)



avail_models_and_members_list = avail_models_and_members_list[avail_models_and_members_list["Rank"]==Rank]

avail_models_and_members_list.drop(columns = ["Var",
                                              "Season",
                                              "Stat",
                                              "NetError",
                                              "Rank"],
                                  inplace = True)

n_ensembles = len(avail_models_and_members_list)

print(avail_models_and_members_list)
print("")
print(" -- Number of Ensembles: ",n_ensembles)

#
######################################################

## Process Masks

maskf = xr.open_dataset(masking_file)

mask  = maskf[masking_variable]

if in_notebook():
    mask.plot()

if (masking_variable == "LOCA2_CLIMDIV"):
    region_keys   = maskf["climdiv"].values
    region_names  = maskf["climdiv_name"].values
    region_states = maskf["climdiv_st_abbrv"].values


print(region_keys)

## Loop Through Ensemble Members



for ensemble in range(n_ensembles):

    model  = avail_models_and_members_list.at[ensemble,  "Model"]
    member = avail_models_and_members_list.at[ensemble, "Member"]
    
    for scenario in Scenarios:

        file_name = "LOCA2-CONUS___" + model +"."+ member + "___" + scenario +".nc"

        file_url  = thredds_root_dir + scenario+ "/"+ file_name

        scenario_inventory  = avail_models_and_members_list.at[ensemble,  scenario]

        if (scenario_inventory != "---"):
            print(file_name)

            loca2f = xr.open_dataset(filename_or_obj = file_url)

            times   = loca2f["time"].to_numpy()
            n_times = len(times)
            
            for k_start in range(0, n_times, 360):
                k_end = min(n_times-1,k_start+360)
                print(times[k_start],times[k_end-1])

                if ("T" in scenario_inventory):
                    in_tasmax = loca2f[:,:,k_start:k_end]

                    print*
                    
                    
            
                          
        


In [73]:
filehist            = "http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Original_CONUS/historical/LOCA2-CONUS___GFDL-ESM4.r1i1p1f1___historical.nc"
histf               = xr.open_dataset(filehist)
calendar_date_hist  = histf["time"].rename("time_hist").rename({'time':'time_hist'})

calendar_date_hists = calendar_date_hist.dt.strftime('%Y-%m-%d %H:%M:%S')
LOCA2_Calendar_Lookup_Table = calendar_date_hists.rename("calendar_date_hist").to_dataset()


filefutr = "http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Original_CONUS/ssp585/LOCA2-CONUS___GFDL-ESM4.r1i1p1f1___ssp585.nc"
futrf    = xr.open_dataset(filefutr)

calendar_date_futr  = futrf["time"].rename("time_futr").rename({'time':'time_futr'})

calendar_date_futrs = calendar_date_futr.dt.strftime('%Y-%m-%d %H:%M:%S')
calendar_date_futrs = calendar_date_futrs.rename("calendar_date_futr").to_dataset()


LOCA2_Calendar_Lookup_Table = LOCA2_Calendar_Lookup_Table.merge(calendar_date_futrs)


#.merge(calendar_date_futr)

display(LOCA2_Calendar_Lookup_Table)
LOCA2_Calendar_Lookup_Table.to_netcdf(path  = "./LOCA2_Calendar_Lookup_Table.nc",
                                     format = "NETCDF3_CLASSIC")

<xarray.Dataset>
Dimensions:             (time_hist: 23741, time_futr: 23741)
Coordinates:
  * time_hist           (time_hist) datetime64[ns] 1950-01-01T12:00:00 ... 20...
  * time_futr           (time_futr) datetime64[ns] 1950-01-01T12:00:00 ... 20...
Data variables:
    calendar_date_hist  (time_hist) object '1950-01-01 12:00:00' ... '2014-12...
    calendar_date_futr  (time_futr) object '1950-01-01 12:00:00' ... '2014-12...